In [2]:
import torch

In [3]:
x = torch.randn(60000,1,224,224)
card =x.numel()
x = torch.where(x>=2.7,x,0)
print(x.shape)
x = x.to_sparse()
x



torch.Size([60000, 1, 224, 224])


tensor(indices=tensor([[    0,     0,     0,  ..., 59999, 59999, 59999],
                       [    0,     0,     0,  ...,     0,     0,     0],
                       [    0,     1,     1,  ...,   220,   223,   223],
                       [   12,   135,   180,  ...,   201,   138,   180]]),
       values=tensor([3.2747, 3.3523, 2.8170,  ..., 3.2728, 2.7796, 3.0577]),
       size=(60000, 1, 224, 224), nnz=10439734, layout=torch.sparse_coo)

In [16]:
(1-x._nnz()/card)*100

99.65306056016156

In [66]:
x.indices().shape

torch.Size([4, 30108023])

In [29]:
import torch
import torch.nn.functional as F


indices = torch.tensor([
    [0, 0, 0, 0], [0, 1, 2, 2], [0, 2, 3, 3]

])
values = torch.tensor([
    1.0, 2.0, 3.0

])
dense_shape = (120000, 1, 448, 448)


sparse_tensor = torch.sparse.FloatTensor(indices.t(), values, torch.Size(dense_shape))


Somme des convolutions sur la dimension 0:
tensor([[[ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0., -1.,  0.,  ...,  0.,  0.,  0.],
         [ 0., -2., -3.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]])

Tenseur sparse global:
tensor(indices=tensor([[0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [0, 1, 1, 1, 2, 2, 3, 3, 2, 2, 3, 3, 4, 4],
                       [1, 1, 1, 3, 1, 3, 1, 3, 2, 4, 2, 4, 2, 4]]),
       values=tensor([ 1.,  1., -2.,  2., -2.,  2., -2.,  2., -3.,  3., -3.,
                       3., -3.,  3.]),
       size=(120000, 1, 448, 448), nnz=14, layout=torch.sparse_coo)


In [43]:
test = torch.zeros(4,1)
test_add =torch.tensor([1,1,1,1])
test_add.shape


torch.Size([4])

In [100]:
test.shape
for i in range(100):
    test = torch.cat((test,test_add.unsqueeze(1)),dim=1)
test.shape

torch.Size([4, 344972])

In [4]:

import torch
import torch.nn.functional as F


indices = x.indices()
print(indices.shape)
values = x.values()
print(values.shape)
dense_shape = [shape for shape in x.size()]
print("dense_shape",dense_shape)

indices =indices.t()
print(dense_shape)
kernel = torch.tensor([
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0]
]).reshape(1, 1, 3, 3)  

convolution_sum = None


global_indices = None
global_values = None


chunk_size = 1000
num_chunks = dense_shape[0] // chunk_size
print(num_chunks)
from tqdm import tqdm

for i in tqdm(range(num_chunks)):
   
    chunk_indices = indices[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]
 
    chunk_indices[:, 0] -= i * chunk_size  
    chunk_values = values[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]

    chunk_sparse_tensor = torch.sparse.FloatTensor(chunk_indices.t(), chunk_values, torch.Size([chunk_size, 1, 224, 224]))


    chunk_dense_tensor = chunk_sparse_tensor.to_dense()

    conv_output = F.conv2d(chunk_dense_tensor, kernel, stride=1, padding=1)

    conv_sum = conv_output.sum(dim=0)

   
    if convolution_sum is None:
        convolution_sum = conv_sum
    else:
        convolution_sum += conv_sum


  
    conv_output = conv_output.to_sparse()
    add_indices=(conv_output.indices() + torch.tensor([[i * chunk_size], [0], [0], [0]]).to(conv_output.indices().device))
    if global_indices is None:
        global_indices= add_indices
    else : 
        global_indices = torch.cat((global_indices,add_indices),dim =1)
    #global_indices.append(conv_output.indices() + torch.tensor([[i * chunk_size], [0], [0], [0]]))#.to(conv_output_sparse.indices().device))
    if global_values is None: 
        global_values = conv_output.values()
        print(global_values.shape)
    else: 
        global_values = torch.cat((global_values,conv_output.values()),dim=0)    
    


global_sparse_tensor = torch.sparse.FloatTensor(global_indices, global_values, torch.Size(dense_shape))


print("Somme des convolutions sur la dimension 0:")
print(convolution_sum)

print("\nTenseur sparse global:")
print(global_sparse_tensor)


torch.Size([4, 10439734])
torch.Size([10439734])
dense_shape [60000, 1, 224, 224]
[60000, 1, 224, 224]
60


  0%|          | 0/60 [00:00<?, ?it/s]/tmp/ipykernel_1846459/757326893.py:39: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  chunk_sparse_tensor = torch.sparse.FloatTensor(chunk_indices.t(), chunk_values, torch.Size([chunk_size, 1, 224, 224]))
  2%|▏         | 1/60 [00:00<00:15,  3.86it/s]

torch.Size([1023916])


100%|██████████| 60/60 [00:19<00:00,  3.08it/s]

Somme des convolutions sur la dimension 0:
tensor([[[-1278.8673,   -16.1458,   -31.9146,  ...,    20.4610,
             83.8704,  1289.2903],
         [-1914.9761,    -5.6840,  -127.9786,  ...,    16.5947,
            113.3512,  1914.1028],
         [-1890.5922,    42.6622,  -143.8203,  ...,    50.8689,
              9.8589,  1898.8342],
         ...,
         [-1915.4642,    54.5628,   115.1492,  ...,   111.6100,
             -4.4437,  1806.7754],
         [-2013.4230,   -25.2011,   186.4000,  ...,     3.1551,
             64.5561,  1840.2511],
         [-1369.9341,    -9.8317,   171.8115,  ...,     4.7723,
             90.7835,  1223.1674]]])

Tenseur sparse global:
tensor(indices=tensor([[    0,     0,     0,  ..., 59999, 59999, 59999],
                       [    0,     0,     0,  ...,     0,     0,     0],
                       [    0,     0,     0,  ...,   223,   223,   223],
                       [   11,    13,   134,  ...,   139,   179,   181]]),
       values=tensor([-3.2747

In [80]:
indices.shape

torch.Size([4, 30106187])

In [ ]:


sparse_tensor = torch.sparse.FloatTensor(indices.t(), values, torch.Size(dense_shape))

kernel = torch.tensor([
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0]
]).reshape(1, 1, 3, 3)  

convolution_sum = None


global_indices = []
global_values = []


chunk_size = 100
num_chunks = dense_shape[0] // chunk_size

for i in range(num_chunks):
   
    chunk_indices = indices[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]
    chunk_indices[:, 0] -= i * chunk_size  
    chunk_values = values[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]

    chunk_sparse_tensor = torch.sparse.FloatTensor(chunk_indices.t(), chunk_values, torch.Size([chunk_size, 1, 224, 224]))


    chunk_dense_tensor = chunk_sparse_tensor.to_dense()

    conv_output = F.conv2d(chunk_dense_tensor, kernel, stride=1, padding=1)

    conv_sum = conv_output.sum(dim=0)

   
    if convolution_sum is None:
        convolution_sum = conv_sum
    else:
        convolution_sum += conv_sum

  
    conv_output_sparse = conv_output.to_sparse()
    global_indices.append(conv_output_sparse.indices() + torch.tensor([[i * chunk_size], [0], [0], [0]]).to(conv_output_sparse.indices().device))
    global_values.append(conv_output_sparse.values())

global_indices = torch.cat(global_indices, dim=1)
global_values = torch.cat(global_values)
global_sparse_tensor = torch.sparse.FloatTensor(global_indices, global_values, torch.Size(dense_shape))


print("Somme des convolutions sur la dimension 0:")
print(convolution_sum)

print("\nTenseur sparse global:")
print(global_sparse_tensor)


In [30]:
print(convolution_sum.shape)

torch.Size([1, 224, 224])


In [5]:
dense =x.to_dense()


conv_output = F.conv2d(dense, kernel, stride=1, padding=1)
del dense
conv_output = conv_output.to_sparse()
conv_output

tensor(indices=tensor([[0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [0, 1, 1, 1, 2, 2, 3, 3, 2, 2, 3, 3, 4, 4],
                       [1, 1, 1, 3, 1, 3, 1, 3, 2, 4, 2, 4, 2, 4]]),
       values=tensor([ 1.,  1., -2.,  2., -2.,  2., -2.,  2., -3.,  3., -3.,
                       3., -3.,  3.]),
       size=(12000, 1, 224, 224), nnz=14, layout=torch.sparse_coo)

In [2]:
!pip install onnx
import onnx

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 88.2 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 KB 44.6 MB/s eta 0:00:00


In [5]:
!pip install onnx2pytorch

from onnx2pytorch import ConvertModel
onnx_model = onnx.load('vgg19-7.onnx')



Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 1.2 MB/s eta 0:00:00


In [6]:
# Convertir le modèle ONNX en modèle PyTorch
pytorch_model = ConvertModel(onnx_model)


Automatic inference of operator: dropout
Automatic inference of operator: dropout


In [26]:
list = []
for param in pytorch_model.named_parameters():
    list.append(param[0])

In [27]:
list

['Conv_vgg0_conv0_fwd.weight',
 'Conv_vgg0_conv0_fwd.bias',
 'Conv_vgg0_conv1_fwd.weight',
 'Conv_vgg0_conv1_fwd.bias',
 'Conv_vgg0_conv2_fwd.weight',
 'Conv_vgg0_conv2_fwd.bias',
 'Conv_vgg0_conv3_fwd.weight',
 'Conv_vgg0_conv3_fwd.bias',
 'Conv_vgg0_conv4_fwd.weight',
 'Conv_vgg0_conv4_fwd.bias',
 'Conv_vgg0_conv5_fwd.weight',
 'Conv_vgg0_conv5_fwd.bias',
 'Conv_vgg0_conv6_fwd.weight',
 'Conv_vgg0_conv6_fwd.bias',
 'Conv_vgg0_conv7_fwd.weight',
 'Conv_vgg0_conv7_fwd.bias',
 'Conv_vgg0_conv8_fwd.weight',
 'Conv_vgg0_conv8_fwd.bias',
 'Conv_vgg0_conv9_fwd.weight',
 'Conv_vgg0_conv9_fwd.bias',
 'Conv_vgg0_conv10_fwd.weight',
 'Conv_vgg0_conv10_fwd.bias',
 'Conv_vgg0_conv11_fwd.weight',
 'Conv_vgg0_conv11_fwd.bias',
 'Conv_vgg0_conv12_fwd.weight',
 'Conv_vgg0_conv12_fwd.bias',
 'Conv_vgg0_conv13_fwd.weight',
 'Conv_vgg0_conv13_fwd.bias',
 'Conv_vgg0_conv14_fwd.weight',
 'Conv_vgg0_conv14_fwd.bias',
 'Conv_vgg0_conv15_fwd.weight',
 'Conv_vgg0_conv15_fwd.bias',
 'Gemm_vgg0_dense0_fwd.weigh

In [20]:
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
image = Image.open('voilier.jpeg')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
   
    
])

image = transform(image)
print(image.shape)
pytorch_model=pytorch_model.eval()
pytorch_model(image.unsqueeze(0))

torch.Size([3, 224, 224])


TypeError: dropout() missing 2 required positional argument: "p", "train"